In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import bs4
import requests

In [62]:
club_data = []
for x in range(1, 33):
    url = 'http://competitie.vttl.be/index.php?menu=1&cur_page='+str(x)
    r  = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")
    table = soup.find('table', attrs={'class':'DBTable'})
    rows = table.find_all('tr')[2:-2]
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        club_data.append([ele for ele in cols if ele])

In [63]:
club_df = pd.DataFrame(club_data)
club_df.columns = ['index', 'club', 'categorie', 'atl lokalen', 'site']
club_df.set_index('index', inplace=True)
club_df.index.name = None

In [65]:
club_df.head()

,club,categorie,atl lokalen,site
000,Indiv. KBTTB/FRBTT,FRBTT,0,-
VTTL,Buitenland,VTTL,0,-
VTTL,Individueel,VTTL,0,None
AF001,Individuel Aile Francophone,AF,0,None
Vl-B000,Individueel Vlaams-Brabant,Vlaams-Brabant & Br.,1,None
Vl-B005,K.T.T.K Pingouin Leuven,Vlaams-Brabant & Br.,1,None
Vl-B161,T.T.C. Den Hoek,Vlaams-Brabant & Br.,1,-
Vl-B182,T.T. Groot-Bijgaarden,Vlaams-Brabant & Br.,2,None
Vl-B213,T.T.C. Humbeek,Vlaams-Brabant & Br.,1,None
Vl-B218,T.T.K Vilvo,Vlaams-Brabant & Br.,1,None


In [ ]:
elo_data = []
for x in range(1, 191):
    url = 'http://competitie.vttl.be/index.php?menu=5&perelo=1&club_id=0&cur_page='+str(x)
    r  = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")
    table = soup.find('table', attrs={'class':'DBTable'})
    rows = table.find_all('tr')[2:-2]
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        elo_data.append([ele for ele in cols if ele])

In [42]:
elo_df = pd.DataFrame(elo_data)
elo_df.columns = ['plaats', 'naam', 'klassement', 'club', 'punten']

In [43]:
elo_df.tail()

,plaats,naam,klassement,club,punten
18983,18923,BOUTEN ALANA,NG,Zonnebeke,0
18984,18923,VANWINNENDAELE CHRISTINE,NG,Ctt Bosquet,0
18985,18923,JAEKEN STAF,NG,Voorshoven,0
18986,18923,CULOT ANAELLE,NG,Palette Rancoise,0
18987,18923,DONNAY DANY,NG,Ans,0
